# 1. Mapa de Casos Encontrados

## Importações

In [1]:
import pandas as pd
import pydeck as pdk

### Lê os ``.csv``

DataSUS

In [ ]:
cols = [
    'CO_MUN_RES','SG_UF', 'ID_MN_RESI'
]
    
datasus_df = pd.read_csv("../data/data_sus/INFLUD22-26-06-2025.csv", sep=";", usecols=cols)

len(datasus_df)

In [3]:
data = datasus_df.rename(columns={
    "CO_MUN_RES": "Codigo_IBGE",
    "SG_UF": "Estado",
    "ID_MN_RESI": "Municipio"
})

data.dropna(subset=['Codigo_IBGE'], inplace=True)
data["Codigo_IBGE"] = data["Codigo_IBGE"].astype(int)

Estações

In [4]:
cols=[
    'Código IBGE do Município', 'Latitude', 'Longitude'
]

stations_df = pd.read_csv("../data/monitor_ar/EstacoesMonitorAr-Nov-2022.csv", sep=";", usecols=cols)


### Ajusta a variável ``coords``

In [ ]:
coords = stations_df.rename(columns = {
    "Código IBGE do Município": "Codigo_IBGE"
})
coords['Codigo_IBGE'] = (coords['Codigo_IBGE'] / 10).astype(int)

coords['Latitude'] = coords['Latitude'].str.replace(',','.')
coords['Longitude'] = coords['Longitude'].str.replace(',','.')

# Converte lat/lon para numérico, tratando erros
coords['Latitude'] = pd.to_numeric(coords['Latitude'], errors='coerce')
coords['Longitude'] = pd.to_numeric(coords['Longitude'], errors='coerce')
coords.dropna(subset=['Latitude', 'Longitude', 'Codigo_IBGE'], inplace=True)

# Cria um DataFrame com UMA coordenada média por IBGE
coords = coords.groupby('Codigo_IBGE', as_index=False).agg(
    Latitude=('Latitude', 'mean'),
    Longitude=('Longitude', 'mean')
)
coords

## 2. Realizar o ``.merge`` (``JOIN``) entre os Dataframes

In [ ]:
df = pd.merge(
    left=data,
    right=coords,
    on='Codigo_IBGE',
    how='inner'
)

df

## 3. Exibir o Mapa de Colunas

In [ ]:
def render_map(df):
    df['coordinates'] = df[['Longitude', 'Latitude']].values.tolist()
    df = df[['Municipio', 'coordinates']]
    df = df.groupby('Municipio').agg(
        coordinates=('coordinates', 'first'),
        count=('Municipio', 'count')
    ).reset_index()

    layer = pdk.Layer(
        'ColumnLayer',
        data=df,
        get_position='coordinates',
        get_elevation='count',
        elevation_scale=10,
        radius=5000,
        get_fill_color=[255, 140, 0, 150],
        pickable=True,
        extruded=True,
    )

    view_state = pdk.ViewState(latitude=-23, longitude=-50, zoom=4, bearing=-45, pitch=45)

    r = pdk.Deck(
        layers=[layer],
        initial_view_state=view_state,
        tooltip={"text": "{Municipio} \nCount: {count}"},
    )
    display(r)

render_map(df)
    